# Digit Recognition

![Digits 0 through 9](imgs/Digits_Cover.png)

Run these blocks in order

In [ ]:
from nn.network import NeuralNetwork
from nn.layer import Layer
from nn.loss import cross_entropy, cross_entropy_derivative # Best for classification yo
from nn.activations import relu, relu_derivative, softmax, softmax_derivative_passthrough
from nn.gd import fit
from utils.loader import load_digits_dataset
from utils.splitter import data_split

import numpy as np

print("Yo. We're going to build and train this neural network right now.")

np.random.seed(11)

nn = NeuralNetwork(loss=cross_entropy, loss_derivative=cross_entropy_derivative)

l2 = 0.001

# 16x16 inputs = 256 features
layer1 = Layer(input_size=256, output_size=128, activation=relu, d_activation=relu_derivative, mask=None, l2_lambda=l2)
layer2 = Layer(input_size=128, output_size=64, activation=relu, d_activation=relu_derivative, mask=None, l2_lambda=l2)
# Use softmax on output; derivative passthrough so CE gradient isn’t multiplied again
layer3 = Layer(input_size=64, output_size=10, activation=softmax, d_activation=softmax_derivative_passthrough, mask=None)

nn.add_layers([layer1, layer2, layer3])


Yo. We're going to build and train this neural network right now.


In [2]:
print("Now we're loading the dataset...")

print("In Jupyter notebooks, it'll be pretty slow, but it shouldn't take more than 30 seconds (still a pretty big dataset to be fair).")

X, Y = load_digits_dataset()  # normalized with scale255

X_train, Y_train, X_val, Y_val, X_test, Y_test = data_split(X, Y, train_ratio=0.8, val_ratio=0.1, seed=42)


Now we're loading the dataset...
In Jupyter notebooks, it'll be pretty slow, but it shouldn't take more than 30 seconds (still a pretty big dataset to be fair).


In [3]:
print("Training...")
# In demos.ipynb
test_error = fit(nn, X_train, Y_train, X_val, Y_val, X_test, Y_test, max_epochs=200, learning_rate=0.005)

print("Done training.")
print(f"Final Test Error: {test_error:.4f}")

nn.save_model("digits_nn_model.pkl")

# Re-test on the actual training data itself to check for overfitting
train_correct = 0
train_total = len(X_train)
for i in range(train_total):
    output = nn.forward(X_train[i])
    predicted_label = np.argmax(output, axis=1)[0]
    actual_label = np.argmax(Y_train[i])

    if train_total % (i + 1) == 0:
        print(f"Train Image {i + 1}: Predicted Label = {predicted_label}, Actual Label = {actual_label}")

    if predicted_label == actual_label:
        train_correct += 1

train_accuracy = (train_correct / train_total * 100) if train_total > 0 else 0.0
print(f"Train Accuracy: {train_accuracy:.2f}%")

Training...
Done training.
Final Test Error: 0.2531
Train Image 1: Predicted Label = 4, Actual Label = 4
Train Image 2: Predicted Label = 8, Actual Label = 8
Train Image 3: Predicted Label = 7, Actual Label = 7
Train Image 6: Predicted Label = 2, Actual Label = 2
Train Image 11: Predicted Label = 4, Actual Label = 4
Train Image 22: Predicted Label = 3, Actual Label = 1
Train Image 33: Predicted Label = 8, Actual Label = 8
Train Image 66: Predicted Label = 7, Actual Label = 7
Train Image 137: Predicted Label = 3, Actual Label = 3
Train Image 274: Predicted Label = 8, Actual Label = 8
Train Image 411: Predicted Label = 4, Actual Label = 4
Train Image 822: Predicted Label = 2, Actual Label = 2
Train Image 1507: Predicted Label = 2, Actual Label = 2
Train Image 3014: Predicted Label = 0, Actual Label = 0
Train Image 4521: Predicted Label = 3, Actual Label = 3
Train Image 9042: Predicted Label = 4, Actual Label = 4
Train Accuracy: 93.90%


In [4]:
print("Now let's actually put this to use. In the data/digits/ folder, I have made a test/ folder with handwritten digits :)")

# Real test set
X, Y = load_digits_dataset("data/digits/tests", target_size=(16,16))
correct = 0
total = len(X)
for i in range(total):
    output = nn.forward(X[i])
    predicted_label = np.argmax(output, axis=1)[0]
    actual_label = np.argmax(Y[i])
    if train_total % (i+1) == 0:
        print(f"Test Image {i + 1}: Predicted Label = {predicted_label}, Actual Label = {actual_label}")

    if predicted_label == actual_label:
        correct += 1

accuracy = (correct / total * 100) if total > 0 else 0.0 # Avoid division by zero although it should never happen
print(f"Test Accuracy: {accuracy:.2f}%")
print("See, this is why we need convolutional neural networks: since Dense networks expect flat 1D inputs .")


Now let's actually put this to use. In the data/digits/ folder, I have made a test/ folder with handwritten digits :)
Test Image 1: Predicted Label = 0, Actual Label = 0
Test Image 2: Predicted Label = 0, Actual Label = 0
Test Image 3: Predicted Label = 0, Actual Label = 0
Test Image 6: Predicted Label = 1, Actual Label = 1
Test Image 11: Predicted Label = 2, Actual Label = 2
Test Image 22: Predicted Label = 8, Actual Label = 5
Test Image 33: Predicted Label = 8, Actual Label = 8
Test Accuracy: 60.00%
See, this is why we need convolutional neural networks: since Dense networks expect flat 1D inputs .


In [5]:
print("Feel free to add more blocks and test your own handwritten digits. Rescale to 16x16 if needed and follow the same template as in here.")
print("You can reuse the model by loading it with NeuralNetwork.load_model(filename).")
print("They are saved as .pkl and HF5 files in the models/ folder.")
print("Note that, for this specific dataset, accuracy is going to be very low since we actually need a CNN. But the Dense network with regularization is still doing a decent job.")

Feel free to add more blocks and test your own handwritten digits. Rescale to 16x16 if needed and follow the same template as in here.
You can reuse the model by loading it with NeuralNetwork.load_model(filename).
They are saved as .pkl and HF5 files in the models/ folder.
Note that, for this specific dataset, accuracy is going to be very low since we actually need a CNN. But the Dense network with regularization is still doing a decent job.
